# Make sure Python Version is 3.8+

In [1]:
import sys
print("User Current Version:-", sys.version)

User Current Version:- 3.10.2 (main, Apr  9 2022, 20:52:01) [Clang 14.0.0 (https://github.com/llvm/llvm-project 78e87970af888bbbd5652c31f3a8


# Some Conversion Helpers

In [9]:
# ascii
def ascii_to_hex(ascii: str): return bytes_to_hex(ascii_to_bytes(ascii))
def ascii_to_bytes(ascii: str): return str.encode(ascii)
# hex
def hex_to_ascii(hex: str): return bytes_to_ascii(hex_to_bytes(hex))
def hex_to_bytes(hex: str): return bytes.fromhex(hex)
# decimal
def hex_to_dec(hex: str): return int(hex, 16)
def dec_to_hex(dec: int): return format(dec, 'x')
# bytes
def bytes_to_ascii(bytes: bytes): return bytes.decode()
def bytes_to_hex(bytes: bytes): return bytes.hex()


def message_to_ints(message: str, modulus: int):
    block_length = len(dec_to_hex(modulus)) // 4
    message_parts = [message[i:i+block_length] for i in range(0, len(message), block_length)]
    return [hex_to_dec(ascii_to_hex(message_part)) for message_part in message_parts]


# Prime Number Generation

In [15]:
# adapted from https://medium.com/@prudywsh/how-to-generate-big-prime-numbers-miller-rabin-49e6e6af32fb
from random import randrange, getrandbits


def is_prime(n, k=128):
    """ Test if a number is prime
        Args:
            n -- int -- the number to test
            k -- int -- the number of tests to do
        return True if n is prime
    """
    if n == 2 or n == 3:
        return True
    if n <= 1 or n % 2 == 0:
        return False
    # find r and s
    s = 0
    r = n - 1
    while r & 1 == 0:
        s += 1
        r //= 2
    # do k tests
    for _ in range(k):
        a = randrange(2, n - 1)
        x = pow(a, r, n)
        if x != 1 and x != n - 1:
            j = 1
            while j < s and x != n - 1:
                x = pow(x, 2, n)
                if x == 1:
                    return False
                j += 1
            if x != n - 1:
                return False
    return True


def generate_prime_candidate(length):
    """ Generate an odd integer randomly
        Args:
            length -- int -- the length of the number to generate, in bits
        return a integer
    """
    # generate random bits
    p = getrandbits(length)
    # apply a mask to set MSB and LSB to 1
    p |= (1 << length - 1) | 1
    return p


def generate_prime_number(length=1024):
    """ Generate a prime
        Args:
            length -- int -- length of the prime to generate, in          bits
        return a prime
    """
    p = 4
    # keep generating while the primality tests fail
    while not is_prime(p, 128):
        p = generate_prime_candidate(length)
    return p


# Demo

In [18]:
print(ascii_to_hex('asdf'))
print(message_to_ints('hello world!', 1_000_000_000))
print('1_000_000_000 prime:', is_prime(1_000_000_000))
print('1_000_000_001 prime:', is_prime(1_000_000_001))

61736466
[26725, 27756, 28448, 30575, 29292, 25633]
1_000_000_000 prime: False
1_000_000_001 prime: False


# RSA Generation and Crypto (not secure, only for educational purposes!)

In [17]:
from typing import List


class RsaCipherText:
    def __init__(self, parts: List[str]):
        self.parts = parts

    def __str__(self):
        return ' '.join(self.parts)


class RsaPublicKey:
    def __init__(self, e, n):
        self.e = e
        self.n = n

    def encrypt(self, plaintext: str):
        parts = message_to_ints(plaintext, self.n)
        encrypted_dec = [pow(part, self.e, self.n) for part in parts]
        encrypted_hex = [dec_to_hex(dec) for dec in encrypted_dec]

        return RsaCipherText(encrypted_hex)


class RsaSecretKey:
    def __init__(self, d, n):
        self.d = d
        self.n = n

    def decrypt(self, ciphertext: RsaCipherText):
        ciphertext_parts_hex = ciphertext.parts
        ciphertext_parts_dec = [hex_to_dec(hex) for hex in ciphertext_parts_hex]
        decrypted_parts_dec = [pow(enc, self.d, self.n) for enc in ciphertext_parts_dec]
        decrypted_parts_hex = [dec_to_hex(dec) for dec in decrypted_parts_dec]

        try:
            decrypted_parts_ascii = [hex_to_ascii(hex) for hex in decrypted_parts_hex]

            return ''.join(decrypted_parts_ascii)
        except Exception as error:
            print('An exception occurred, probably due to bad key: {}'.format(error))
            return None


def generate_rsa_keypair(length=64):
    """ Generate an RSA key pair
        Args:
            length -- int -- RSA key size (1024 or higher recommended, lower values only for educational purposes)
        return key pair
    """
    prime_p = generate_prime_number(length=length//2)  # discarded
    prime_q = generate_prime_number(length=length//2)  # discarded
    euler_totient = (prime_p - 1) * (prime_q - 1)  # discarded

    modulus_n = prime_p * prime_q  # public
    pk_exponent_e = 65537  # public
    sk_exponent_d = pow(pk_exponent_e, -1, euler_totient)  # secret

    sk = RsaSecretKey(sk_exponent_d, modulus_n)
    pk = RsaPublicKey(pk_exponent_e, modulus_n)

    return sk, pk


# RSA Demo

In [19]:
test_phrase = 'Lorem ipsum dolor sit amet'
sk, pk = generate_rsa_keypair(128)

print('generated key pair')
print('----')
print('public key (used for encryption):')
print('exponent e =', pk.e)
print('modulus  n =', pk.n)
print('private key (used for decryption):')
print('exponent d =', sk.d)
print('modulus  n =', sk.n)
print('----')

ciphertext = pk.encrypt(test_phrase)
print('encrypted  "', test_phrase, '" to "', ciphertext, '"')

deciphered = sk.decrypt(ciphertext)
print('deciphered "', ciphertext, '" to "', deciphered, '"')

generated key pair
----
public key (used for encryption):
exponent e = 65537
modulus  n = 187718935028675437110006705057136780619
private key (used for decryption):
exponent d = 23269735083585749284476186022526993153
modulus  n = 187718935028675437110006705057136780619
----
encrypted  " Lorem ipsum dolor sit amet " to " fdf4c829bc12ece178c8856b82833d2 43d2120d158410ef56c26b9909220868 60ac5a067bbc021d73f3b124d9975a0a c83448872ec77026dda963946f45207 "
deciphered " fdf4c829bc12ece178c8856b82833d2 43d2120d158410ef56c26b9909220868 60ac5a067bbc021d73f3b124d9975a0a c83448872ec77026dda963946f45207 " to " Lorem ipsum dolor sit amet "


----
# RSA Challenges

In [21]:
ciphertexts = {
    64: [
        RsaCipherText('1a07d1e642de1e36 3421c83509d47caf 47906ac2b835423f 58c23d1cbdcf98f6 14cdb0e4a4a1a58f'.split(' ')),
        RsaCipherText('4eee78f8b62fd9a8 554ce52357b79cb1 2a8f7d80c9c12151 1f7ca1d9d74d3d99 71e4d893c1e2cdc 7f2d12b5954ad586 307e07227bbf394d 7bc97b0d073d9c65'.split(' ')),
        RsaCipherText('915dab39e33c2254 36af3b63724fef0d eef7aa7240698f6 97b6f982593bf212 1991c202c109b709'.split(' ')),
        RsaCipherText('60a58f8e9bdd060f 1b0e7233bdc4352e 165b3631bf05ac03 22f40559084a96ca 2aae5afccedd3ca3 43e5945fbeed1dd3 60a58f8e9bdd060f 3510840210db9f26 63a6350d19263352 2fb5239776c019af 163256293a6b4156 62c3add9ef3b4322 4297a5baa1e3b8a5 279ffe4b4d98427b 56d42c6995cb6a0e'.split(' '))
    ],
    128: [
        RsaCipherText('44a88cfca3684c58a572f7bb5a3ef248 75c8da5c5ff720081364e1986c448c35 4004069f6efd6e072f0a232fe9ef660a 505d04c0142f2bd391d98b1fdbe23dc4 31c6cef38df81318759ec0ea41e2d5ad'.split(' ')),
        RsaCipherText('f7cb35bba40bd99e566742941f31df 5990fed34244af893ecf5b4809e210c9 117493545a6f065e16f359df73e91ebd'.split(' ')),
        RsaCipherText('5232ceca3cca9d575fa5744d429ca945 34729a4e9ae525c83d58f97f1204be6c 5402a9076cfcd525e204ca81de13672d 6cfd8660758727a4d0f7e39901377c96 2dcad4d39a287b15654010786263be11'.split(' ')),
        RsaCipherText('733f71c920072be41d6c6f416686e63f 84133f18b7f683a85a975f6b94fb2798 3012aff76f1633ddc01bd8942c793acd'.split(' '))
    ],
    256: [
        RsaCipherText('af9014c5b8f554cd0acf193a99982e08cf8e48d40b082817622851eb36e84883 627c8c1509380ec1f1d83e928523142e8ce40f448c95ce1b10ed33766852a87a'.split(' ')),
        RsaCipherText('50ac17d87d53440dec08909681fe87348911cbab9fc3b9130ebda1d5e7bda376 78f59177f458fedb70df73a967324b85752d48ddea9c981fbd3b00923ca8b190 76bdab842c8561a1c0ab1502d7b764c8f4419161ed9f8e402182b501351e2dac 1d54221aaa3e847264e6ce3233d11bc7c5fc080b88da2d18c43ab65e3e103cc9 60af0f5107b81e0a9ad2843b83d0714f5149f18368344bff3dfc8549e3e20ce0 19bc421db03292b63fdec6869058df2ffdc10cd3d2f31d08da670dc9a2d86e85 1ae05d16ff1e13d1b8a0a2f79a9932d5c8e3a85a8641bf873b23deab47c98673 5c326ae7be14a9c71223ef53601281fae93cc07fb3b7072e708e896ab8803d3a 46d702469bb69d69a435b1b98c635a356d8933ca9a4daebecd7b87aae07ea2bc'.split(' ')),
        RsaCipherText('5200ed7ef56080299d8758f42393da621f624836f440edfbe6d8c93d0bef6e15 1669a39b9e224462a33da3b3879d1d1705b95c33fe9e98872d5bb34eb518d1a2 537744a5ccc9a347145967531c0fa111fbd577afaa2aaf927939327f232c9f9c 3be2358b1e19b93df271c9d54d353f6b2c061c2195cb408548045a26a68a69d5'.split(' ')),
        RsaCipherText('23ce3546d99ad34c5be56d72fef5cd9b9c8a45d4618e6ee314254cf47bd0d0bb'.split(' '))
    ],
    512: [
        RsaCipherText('c6bb986358773d9beee91c7b480a8e225b3579bf15180be5492b903933be02a37cb1ad0c66fc2fd3116bda1c9d488098f176686254813140a7efe09116cf64a 4b1b28047f8cec184cfce416219de224cd0461585ebb857d752cdd15520c31aa97c395f7fb79d481d676a4b06744e0570acd9ac9098da16671024cb79256a2a7'.split(' ')),
        RsaCipherText('34bf8b2b8f12794d4a098a96296f40a467145b9e3f613662926cbe12335c64fee33669ea216ade7724d998bf25e7e54a681521bff3a39062b1bd9ecfa154feec'.split(' ')),
        RsaCipherText('34b433caf7a62ebd5a2ecfc9c5e2c7576ec1e4fb9179ba4335c4240657c35bd400b0f13a1c87a88972f060d2289947cb2bc8693ec85c4ff86bd6d0c2c357cd09 2b1e9a92af280e7ad167092d464207e65eea1ec9d1bc2fd5a4ac9f65aeac5ab2d109d726a1026260d97da3052b1fd9fb039f72075ac1083dbf1dd1583705d93a'.split(' ')),
        RsaCipherText('71874aa41144ee1afb42456b1f8aeafb775a069de706e756940b5adff87ef7fddc41e467b855ac45b2ae43660098bab0e896db8076de46d9ffa097f7d54e99b0'.split(' '))
    ],
    1024: [
        RsaCipherText('238c9b9aaa5a08955a884895e4646383ba8e70f9da7c1185a51132c78286f299bf7e248e78c2c67427807646f6e3bff5111144c8e1bed5f32584b974dfa1ae5886a0ad7affc7401b6ccea8c5347c041993b66b0cf2b3c9806473e2fc25c9db08fee830922a55bff33cffb45bbc3b7aad0df380eed7f6e3ea8a0a286743850a00'.split(' ')),
        RsaCipherText('450c1103f554d78de82e61e749a18793d13fe6096498aa096bc4cf0bd5c482fa480c0c7b3a00d139723269bc37ec4b6a3263ee26400b187177e05115ba9554aa53560d9f7df5e84d74c3d9ad283df2992c1cbe6e40ff4ebd3281029cac2264f33f119320c211b684ce59e85fe6c14fdd07d818ddab34051d61a442d63fb42d9bb'.split(' ')),
        RsaCipherText('1dfba267efefc2f0210fadac73fb723b2238b9916655b461a4f167658e3f8c7c7c2de1079c8b3d55f61737a650b5259e7baecfa59b087b623987f2f6c3c2e933442731aa593913dd7857c3c1c8a08285fecf74d870de8afcb1ca86dbd4cd26e7c0e1f4880f8dee3eaaedd26934d02905ce67e07433a39b2e93368b1a0fc9a88e8'.split(' ')),
        RsaCipherText('4490208324028c9a8faacc507ff9cdfef77b813a332f9935c9df8b61365833f954d87a1d8f15174c2d2eb771de99bbca1a2c94b425cddfa978915100ad59aba5ebf58d252527917e0002d53aac9541ea6f64bccb12fd6ae2554a51b601735ba8338c6c113e3457c5479c280ff3f2551e8e27d5e865e6b87affac6329d0bd7694a'.split(' '))
    ]
}

rsa_keys = {
    # sympy.ntheory.factorint and primefac.primefac can do this
    64: [
        RsaPublicKey(e=65537, n=6660964557283730119),
        RsaPublicKey(e=65537, n=9714197806864012093),
        RsaPublicKey(e=65537, n=12128303427012391181),
        RsaPublicKey(e=65537, n=7264648477834627853)
    ],
    # sympy.ntheory.factorint and especially primefac.primefac will start having issues here
    # WolframAlpha can do this quickly, but where's the fun in that
    128: [
        RsaPublicKey(e=65537, n=237894710146046503794834253572456500677),
        RsaPublicKey(e=65537, n=129640883716406873556243653634291956149),
        RsaPublicKey(e=65537, n=172724129383842054897551241391589653769),
        RsaPublicKey(e=65537, n=246073823402909187169525808947714866463)
    ],
    # probably not a good idea to attempt this with sympy.ntheory.factorint
    # ~157 is the limit for WolframAlpha
    # check out the following links instead to bring out the big guns:
    # http://gilchrist.ca/jeff/factoring/nfs_beginners_guide.html
    # https://github.com/truongkma/ctf-tools/blob/master/factmsieve.py
    # note of caution: this challenge takes the most time, including setup and running the sieve at least an hour or so
    256: [
        RsaPublicKey(e=65537, n=80653121571881396176487547859548992154653658177920429659095844537393715553287),
        RsaPublicKey(e=65537, n=56367746323808765701701774043191184766245334323024577984414926296878083778771),
        RsaPublicKey(e=65537, n=74157334360821273857040735435986955425994203755710637894592828327688421012349),
        RsaPublicKey(e=65537, n=35756820799419991114643714201785739656292560715402729333427109774461198433387)
    ],
    # this requires substantially less setup than the 256-block, but you'll need to get crafty for this one.
    # hint: read up on gcd attacks
    512: [
        RsaPublicKey(
            e=65537,
            n=4667556790695065213243698193091437335064489351259616106397803257451176679116795569795548416954453154529519185357017865681283372480309863060487002340065479
        ),
        RsaPublicKey(
            e=65537,
            n=6204956559339069410352812822961765313985342951806425001602965550418315354975834088081354377881639170854807583572476956441269396436677232108288554185941201
        ),
        RsaPublicKey(
            e=65537,
            n=6067664567628593441616234382549274983301945399676127268483215898100372803527007944882904166130794928407589810594047175872933320624972981189152831959253373
        ),
        RsaPublicKey(
            e=65537,
            n=5970005732560416127865497039736411869540448056616985051264278065791556260779196996889705774067755801237903204319660956570979422790295302015391293043015097
        )
    ],
    # this challenge is somewhat similar to the 512-block in the sense that there's a trick to it.
    # hint: check out wiener's attack and broadcast attack
    1024: [
        RsaPublicKey(
            e=56172436577459725698934391359139104915041430213184221292301658571726414059411889155782982024019814564512291421932489731563519296372873415080546379424619308859152360214209740169135159761234894923144971372974038021945201954600238994209605035703317119192844975463915465725406543097929017637859019950590916533609,
            n=77531969503748326589677418948315140870584015245386763633241518845356850979564402923266696704186567270006361208862086254527576010412135230279553684940635956656649728134893874567619948675304052482720430367748612708917105846534082863042823913166120865362252479206576942147071396319459112580853771742537940112457
        ),
        RsaPublicKey(
            e=3,
            n=1001191535967882284769094654562963158339094991366537360172618359025855097846977704928598237040115495676223744383629803332394884046043603063054821999994629411352862317941517957323746992871914047324555019615398720677218748535278252779545622933662625193622517947605928420931496443792865516592262228294965047903627
        ),
        RsaPublicKey(
            e=3,
            n=405864605704280029572517043538873770190562953923346989456102827133294619540434679181357855400199671537151039095796094162418263148474324455458511633891792967156338297585653540910958574924436510557629146762715107527852413979916669819333765187674010542434580990241759130158992365304284892615408513239024879592309
        ),
        RsaPublicKey(
            e=3,
            n=1204664380009414697639782865058772653140636684336678901863196025928054706723976869222235722439176825580211657044153004521482757717615318907205106770256270292154250168657084197056536811063984234635803887040926920542363612936352393496049379544437329226857538524494283148837536712608224655107228808472106636903723
        )
    ]
}

# RSA Cracking Attempt

In [26]:
ciphertext_64_1 = ciphertexts[64][0]
public_key_64_1 = rsa_keys[64][0]

# TODO: figure out d
d = 1337
secret_key_64_1 = RsaSecretKey(d=d, n=public_key_64_1.n)

deciphered = secret_key_64_1.decrypt(ciphertext_64_1)
if deciphered is not None:
    print('deciphered "', ciphertext_64_1, '" to:')
    print(deciphered)
else:
    print('d =', d, 'is not the right key')

An exception occurred, probably due to bad key: 'utf-8' codec can't decode byte 0xf7 in position 3: invalid start byte
d = 5024816920096375840 is not the right key
